In [ ]:
!pip install kaggle


In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/julian3833/jigsaw-toxic-comment-classification-challenge")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sutharimanikanta
Your Kaggle Key: ··········


100%|██████████| 53.4M/53.4M [00:00<00:00, 90.3MB/s]


In [ ]:
#https://www.kaggle.com/code/iabhishekmaurya/transfer-learning-resnet50v2


In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
train=pd.read_csv("/content/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
test=pd.read_csv("/content/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
samp_sub=pd.read_csv("/content/jigsaw-toxic-comment-classification-challenge/sample_submission.csv")
print(f"Train Data Dimensions are {train.shape}")
print(f"Test Data Dimensions are {test.shape}")
print(f"Sample Submission Dimensions are {samp_sub.shape}")
train.head(3)

Train Data Dimensions are (153164, 7)
Test Data Dimensions are (153164, 7)
Sample Submission Dimensions are (153164, 7)


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1


In [ ]:
import plotly.express as px
df1=pd.DataFrame(train[train.columns[2:]].sum(axis=0)).reset_index()
df1["Label"]=df1["index"]
df1["Count of Comments"]=df1[0]
df1=df1.sort_values(0, ascending=False)
fig = px.bar(df1, x="Label", y="Count of Comments", title="No. of comments per label",color="Count of Comments",text="Count of Comments")
fig.update_traces( textposition='outside')
fig.show()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
train_data="/content/jigsaw-toxic-comment-classification-challenge/train.csv"
test_data="/content/jigsaw-toxic-comment-classification-challenge/test.csv"

In [ ]:
from tensorflow.keras import layers

# Create base model
input_shape = (224, 224,3)
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = layers.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(train_data))(x) # want one output neuron per class
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("sigmoid", dtype=tf.float32, name="sigmoid_float32")(x)
model = tf.keras.Model(inputs, outputs)
# Compile the model
model.compile(loss="binary_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
# Assuming train_data and test_data are instances of tf.data.Dataset

mobilenet_history1 = model.fit(train_data,
                               epochs=5,
                               validation_data=test_data)



IndexError: ignored